### Imports
First, make sure the following packages are installed per the [pandas documentation](https://pandas.pydata.org/docs/getting_started/install.html#install-recommended-dependencies):
- `bottleneck`
- `numexpr`

In [1]:
# standard lib
import functools

# third party
import numpy as np
import pandas as pd

### Create a random dataframe
The # of rows in the dataframe can be adjusted

In [2]:
df_rows = 1000

# equivalent to setting seed
rng = np.random.default_rng(3737)

df = pd.DataFrame({
    "var1": rng.integers(1, 100, df_rows),
    "var2": rng.integers(1, 100, df_rows),
    "var3": rng.integers(1, 100, df_rows),
    "total": rng.integers(101, 200, df_rows)
})

In [3]:
df.head()

,var1,var2,var3,total
0,28,64,43,114
1,84,74,9,153
2,27,59,35,144
3,46,26,74,151
4,94,90,63,185


### Timing
Dictionary to store timing

In [4]:
timing = dict()

### Base
Base case with just simple `for` loops

In [5]:
%%timeit -o
sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            df_temp = df[(df["var1"] >= i) & (df["var2"] >= j) & (df["var3"] >= k)]
            sum_list.append(df_temp["total"].sum())

6.67 s ± 219 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 6.67 s ± 219 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [6]:
timing.update({"base": _})

### Base - no intermediate dataframe
Do not create an intermediate dataframe


In [7]:
%%timeit -o
sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(df[(df["var1"] >= i) & (df["var2"] >= j) & (df["var3"] >= k)]["total"].sum())

6.45 s ± 371 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 6.45 s ± 371 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [8]:
timing.update({"base_no_int_df": _})

### numpy with `where`
Utilize `where` and numpy (utilizing numpy vectorizes operations)

In [9]:
%%timeit -o
sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(
                np.sum(
                    (np.where(df["var1"] >= i, 1, 0) & np.where(df["var2"] >= j, 1, 0) & np.where(df["var3"] >= k, 1, 0)) * df["total"]
                )
            )

4.37 s ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 4.37 s ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [10]:
timing.update({"np_where": _})

### numpy with `where` and `to_numpy`
Utilize `where` and numpy but convert cols to `numpy` arrays first

In [11]:
%%timeit -o
sum_list=[]

var1_array = df["var1"].to_numpy()
var2_array = df["var2"].to_numpy()
var3_array = df["var3"].to_numpy()
total_array = df["total"].to_numpy()

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(
                np.sum(
                    (np.where(var1_array >= i, 1, 0) & np.where(var2_array >= j, 1, 0) & np.where(var3_array >= k, 1, 0)) * total_array
                )
            )

299 ms ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 299 ms ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [12]:
timing.update({"np_where_to_numpy": _})

### Cache function
Utilize function caching (and partial application)

In [13]:
%%timeit -o

var1_array = df["var1"].to_numpy()
var2_array = df["var2"].to_numpy()
var3_array = df["var3"].to_numpy()
total_array = df["total"].to_numpy()

def filter_and_mult_total(a1, a2, a3, at, i, j, k):
    return np.sum((np.where(a1 >= i, 1, 0) & np.where(a2 >= j, 1, 0) & np.where(a3 >= k, 1, 0)) * at)

# partial application
# necessary as ndarrays are not hashable
filter_and_mult_total_partial = functools.partial(filter_and_mult_total, var1_array, var2_array, var3_array, total_array)

# cache
@functools.cache
def filter_and_mult_total_cache(i, j, k):
    filter_and_mult_total_partial(i, j, k)

sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(filter_and_mult_total_partial(i, j, k))

279 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 279 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [14]:
timing.update({"cache_func": _})

### Cache results of `where` statements
Cache results of `where` statements rather than caching the function call

Basically a form of [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) where we save intermediate results and just look them up if they already exist; Python dictionary lookups are fast

In [15]:
%%timeit -o
sum_list=[]

var1_array = df["var1"].to_numpy()
var2_array = df["var2"].to_numpy()
var3_array = df["var3"].to_numpy()
total_array = df["total"].to_numpy()

i_dict = dict()
j_dict = dict()
k_dict = dict()

for i in range(1, 100, 5):
    # not technically caching but no need to recalculate if i is not changing
    # setdefault returns...
    #   - if key exists, return value
    #   - if key does not exist, add to dictionary, and then return the 2nd argument
    i_result = i_dict.setdefault(i, np.where(var1_array >= i, 1, 0))
    for j in range(1, 100, 5):
        j_result = j_dict.setdefault(j, np.where(var2_array >= j, 1, 0))
        for k in range(1, 100, 5):
            k_result = k_dict.setdefault(k, np.where(var3_array >= k, 1, 0))
            sum_list.append(
                np.sum(
                    (i_result & j_result & k_result) * total_array
                )
            )

184 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 184 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [16]:
timing.update({"cache_where": _})

### Cache results of `where` statements and precreate lookup dict
Cache results of `where` statements

Basically a form of [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) where we save intermediate results and just look them up if they already exist

Create full dicts and only perform lookup in the `for` loops rather than using `setdefault`

In [17]:
%%timeit -o
sum_list=[]

var1_array = df["var1"].to_numpy()
var2_array = df["var2"].to_numpy()
var3_array = df["var3"].to_numpy()
total_array = df["total"].to_numpy()

i_dict = {i: np.where(var1_array >= i, 1, 0) for i in range(1, 100, 5)}
j_dict = {j: np.where(var2_array >= j, 1, 0) for j in range(1, 100, 5)}
k_dict = {k: np.where(var3_array >= k, 1, 0) for k in range(1, 100, 5)}

for i in range(1, 100, 5):
    # not technically caching but no need to recalculate if i is not changing
    i_result = i_dict.get(i)
    for j in range(1, 100, 5):
        j_result = j_dict.get(j)
        for k in range(1, 100, 5):
            k_result = k_dict.get(k)
            sum_list.append(
                np.sum(
                    (i_result & j_result & k_result) * total_array
                )
            )

103 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 103 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [18]:
timing.update({"cache_where_precreate_dict": _})

### Speedup
What is the final speedup?

In [19]:
# speedup over base case
for k,v in timing.items():
    print(f"{k}\n{'_'*len(k)}")
    print(f"avg.  speed up over base: {timing.get('base').average/timing.get(k).average:.2f} x")
    print(f"worst speed up over base: {timing.get('base').worst/timing.get(k).worst:.2f} x")
    print(f"best  speed up over base: {timing.get('base').best/timing.get(k).best:.2f} x\n")

base
____
avg.  speed up over base: 1.00 x
worst speed up over base: 1.00 x
best  speed up over base: 1.00 x

base_no_int_df
______________
avg.  speed up over base: 1.04 x
worst speed up over base: 1.00 x
best  speed up over base: 1.07 x

np_where
________
avg.  speed up over base: 1.53 x
worst speed up over base: 1.59 x
best  speed up over base: 1.46 x

np_where_to_numpy
_________________
avg.  speed up over base: 22.32 x
worst speed up over base: 18.87 x
best  speed up over base: 23.00 x

cache_func
__________
avg.  speed up over base: 23.88 x
worst speed up over base: 24.73 x
best  speed up over base: 22.80 x

cache_where
___________
avg.  speed up over base: 36.23 x
worst speed up over base: 32.20 x
best  speed up over base: 37.02 x

cache_where_precreate_dict
__________________________
avg.  speed up over base: 64.64 x
worst speed up over base: 65.85 x
best  speed up over base: 62.20 x

